<a href="https://colab.research.google.com/github/NathanielRose/eeg-294/blob/master/Gumpy_Testing_SH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [104]:
!pip install git+git://github.com/gumpy-bci/gumpy.git

  Cloning git://github.com/gumpy-bci/gumpy.git to /tmp/pip-req-build-ss5prc6h
  Running command git clone -q git://github.com/gumpy-bci/gumpy.git /tmp/pip-req-build-ss5prc6h
  Created wheel for gumpy: filename=gumpy-0.5.0-cp36-none-any.whl size=32999 sha256=fe7a880c723a8108294054e87797aaa26c3d5ec515f68c4f93aecc6c6de65a3b
  Stored in directory: /tmp/pip-ephem-wheel-cache-5k4ga4my/wheels/cf/ee/4d/dfa04a0d04ffb1ca36f436f3fef4817e9e23bfec03dd772913
Successfully built gumpy


In [1]:
%reset
%matplotlib inline

from __future__ import print_function
import os; os.environ["THEANO_FLAGS"] = "device=gpu0"
import os.path
from datetime import datetime
import sys

# import gumpy
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import pandas as pd

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)


Mounted at /content/drive


In [3]:
working_dir_sh = '/content/drive/My Drive/Projects_Ongoing/classes_and_workshops/CS294-082/eeg-294/'
working_dir = working_dir_sh
sys.path.append(working_dir + 'gumpy-deeplearning')
import models
sys.path.append(working_dir + 'gumpy-deeplearning/models/')
import utils
sys.path.append(working_dir + 'gumpy/gumpy/')
import gumpy

Using TensorFlow backend.


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
!git clone https://github.com/NathanielRose/eeg-294.git

fatal: destination path 'eeg-294' already exists and is not an empty directory.


In [0]:
sys.path.append('gumpy/gumpy/')
import gumpy

In [7]:
gumpy.data

<module 'gumpy.data' from '/usr/local/lib/python3.6/dist-packages/gumpy/data/__init__.py'>

# Get Data

In [0]:
data_dir = working_dir + 'data/'
subject = 's4'


In [0]:
gumpy.signal.process

AttributeError: ignored